# DL Assignment 2

Ishaan Reddy
SCDS
210200019

In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image

2.10.1


In [2]:
imgFiles = glob.glob(r'C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy/*/*.jpg')


for item in imgFiles:
    print(item)

C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (1).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (10).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (11).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (12).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (13).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (14).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (15).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (16).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (17).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning

In [6]:
X = []
y = []

for fName in imgFiles:

  # Prepare the dataset and populate X and y
    X_i = Image.open(fName) 
    X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning
    X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 (not stretching, only scaling)

    X.append(X_i)

    label = fName.split("\\")
    y_i = label[-2] 

    y.append(y_i)


In [7]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)

print(set(y))
print(lEncoder.classes_)

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

{0, 1, 2, 3, 4, 5, 6}
['african-wildcat' 'blackfoot-cat' 'chinese-mountain-cat' 'domestic-cat'
 'european-wildcat' 'jungle-cat' 'sand-cat']
(519, 299, 299, 3)
(519,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))

X_train_shape: (389, 299, 299, 3)
X_test_shape: (130, 299, 299, 3)


In [9]:
mu = X_train.mean()
std = X_train.std()

X_train_std = (X_train-mu)/std
X_test_std = (X_test-mu)/std

In [10]:
class_counts = dict()

# Count images for each class
for file_path in imgFiles:
    class_name = file_path.split("\\")[-2]
    if class_name not in class_counts:
        class_counts[class_name] = 1
    else:
        class_counts[class_name] += 1

for class_name, count in class_counts.items():
    print(f"Class: {class_name}, Count: {count}")

Class: african-wildcat, Count: 91
Class: blackfoot-cat, Count: 79
Class: chinese-mountain-cat, Count: 42
Class: domestic-cat, Count: 64
Class: european-wildcat, Count: 85
Class: jungle-cat, Count: 86
Class: sand-cat, Count: 72


a. Total number of images in the dataset = 519

b. Number of images present per class
    Class: african-wildcat, Count: 91
    Class: blackfoot-cat, Count: 79
    Class: chinese-mountain-cat, Count: 42
    Class: domestic-cat, Count: 64
    Class: european-wildcat, Count: 85
    Class: jungle-cat, Count: 86
    Class: sand-cat, Count: 72 
    
c. Total number of images used for training and testing.
    Train = 389
    Test = 130

## Transfer Learning

### Model 1: InceptionV3

In [18]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D


base_model_1 = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


for layer in base_model_1.layers:
    layer.trainable = False


x = base_model_1.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.25)(x)
output_1 = Dense(7, activation='softmax')(x) 


model_TL1 = Model(inputs=[base_model_1.input], outputs=[output_1])

In [19]:
model_TL1.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

callbacks_TL1 = [
             keras.callbacks.ModelCheckpoint("bestTL.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)]

history_TL1 = model_TL1.fit(x = X_train_std, y = y_train, epochs=10,
                          validation_split=0.1, batch_size=16, callbacks=callbacks_TL1)

Epoch 1/10
22/22 [==============================] - 60s 2s/step - loss: 2.0085 - accuracy: 0.3486 - val_loss: 1.5307 - val_accuracy: 0.5128
Epoch 2/10
22/22 [==============================] - 33s 2s/step - loss: 0.9319 - accuracy: 0.6714 - val_loss: 1.3180 - val_accuracy: 0.5385
Epoch 3/10
22/22 [==============================] - 27s 1s/step - loss: 0.6112 - accuracy: 0.7943 - val_loss: 1.2699 - val_accuracy: 0.5897
Epoch 4/10
22/22 [==============================] - 21s 947ms/step - loss: 0.4425 - accuracy: 0.8457 - val_loss: 1.1116 - val_accuracy: 0.5897
Epoch 5/10
22/22 [==============================] - 23s 1s/step - loss: 0.4110 - accuracy: 0.8514 - val_loss: 1.0217 - val_accuracy: 0.6154
Epoch 6/10
22/22 [==============================] - 27s 1s/step - loss: 0.2903 - accuracy: 0.9000 - val_loss: 0.9727 - val_accuracy: 0.6154
Epoch 7/10
22/22 [==============================] - 22s 988ms/step - loss: 0.2479 - accuracy: 0.9286 - val_loss: 0.9183 - val_accuracy: 0.6410
Epoch 8/10
22/

### Evaluate the model 

In [20]:
testLoss_TL, testAccuracy_TL = model_TL1.evaluate(x = X_test_std, y = y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 8s 1s/step - loss: 0.7480 - accuracy: 0.7077
Test-loss: 0.748036, Test-accuracy: 0.707692


### Updating with the best weights 

In [21]:
model_TL1.load_weights("bestTL.h5")

testLoss_TL, testAccuracy_TL = model_TL1.evaluate(x = X_test_std, y = y_test)

print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 7s 1s/step - loss: 0.8093 - accuracy: 0.7000
Test-loss: 0.809321, Test-accuracy: 0.700000


In [22]:
model_TL1.save('/content/drive/MyDrive/DL_Models/01_Xception_TransferLearning_Best_Model.h5')

In [23]:
from numpy import save

save('/content/drive/MyDrive/DL_Models/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/DL_Models/X_test_std.npy', X_test_std)

save('/content/drive/MyDrive/DL_Models/y_train.npy', y_train)
save('/content/drive/MyDrive/DL_Models/y_test.npy', y_test)